# Merging, joining and concatenating

En este notebook vamos a ver como podemos trabajar dataset que combinan multiples fuentes de datos. Imaginemos una base de datos de un restaurante donde se registran por un lado las ventas, por otro lado los clientes y por otro tenemos las comidas que se venden. Para poder realizar un analisis de datos es necesario juntar las fuentes de datos en una central. Esto lo vamos a realizar a traves de las operaciones *merge*, *concatenate* y *join*.

Vamos a utilizar informacion acerca de un restaurante distribuida en diferente *datasources*. Por un lado tenemos las ventas de dos semanas consecutivas en dos ficheros separados. Un fichero clientes, un fichero de comidas y uno de satisfaccion.

In [ ]:
import pandas as pd

In [ ]:
# verifica la ruta de carga de tu fichero.
df_week1 = pd.read_csv("pandas/Restaurant - Week 1 Sales.csv") # 1a semana de ventas
df_week2 = pd.read_csv("pandas/Restaurant - Week 2 Sales.csv") # 2a semana de ventas
df_week1.head()

Aqui tenemos una combinacion de un identificador de cliente y que identificador producto compro en el restaurante durante dos semanas diferentes (veremos mas adelante como saber que significa cada identificador). De este modo tenemos almacenado quien vino a comer y que ha comido.

A continuacion cargamos el fichero clientes que nos enlaza el identificador con los datos de cada cliente. Esto nos va a permitir obtener informacion acerca de los clientes que compraron en nuestro restaurante.

In [ ]:
# el ID de este fichero enlaza con el Customer ID del fichero de ventas
df_customers = pd.read_csv("pandas/Restaurant - Customers.csv")
df_customers.head()

Y finalmente tenemos informacion de la comida vendida. De forma similar al fichero clientes, este fichero enlaza con los campos *Food ID* de las ventas con la comida consumida en el restaurante.

In [ ]:
# el ID de este fichero enlaza con el food ID del fichero de ventas 
# para obtener una descripcion de la comida
df_foods = pd.read_csv("pandas/Restaurant - Foods.csv")
df_foods.head()

## El método `pd.concat()`

Este metodo nos va a permitir concatenar dos dataframes que son identicos. Por ejemplo, en nuestro caso podemos concatenar las dos semanas de ventas para tenerlas juntas en un solo DataFrame. Esto se realiza a traves del metodo *pd.concat()* al que se le pasa una lista con los DataFrames a concatenar.

In [ ]:
print('Number of sales week 1:')
print(len(df_week1))
print('Number of sales week 2:')
print(len(df_week2))
print('Number of sales both weeks together:')
print(len(pd.concat([df_week1, df_week2])))

Al concatenar Pandas pega un DataFrame detras del otro y mantiene los indices intactos. Si queremos que se cree un nuevo DataFrame con nuevos indices (desde 0 hasta n-1) hay que pasar el parametro *ignore_index*.

In [ ]:
# concatenamos dos semanas en un DataFrame y crea nuevo indice
df_two_weeks = pd.concat([df_week1, df_week2], ignore_index=True)
df_two_weeks.head()

Esto esta bien pero si observas, al concatenar perdemos la referencia de que semana viene cada uno de los datos. Puede ser interesante guardar la informacion de donde vienen cada uno de los datos que concatenamos. Imagina concatenar una serie de experimentos, te puede interesar guardar a que experimento pertenece cada uno de los datos del nuevo DataFrame. Esto es muy sencillo de hacer usando el parametro *keys* que el metodo *concat* tiene.

In [ ]:
df_two_weeks = pd.concat([df_week1, df_week2], keys=['week1', 'week2'])
df_two_weeks.head()

Podras notar que hemos creado un nuevo MultinIndex DataFrame en el que tenemos las semanas. Acceder a los registros de cada una de las semanas es muy sencillo a traves del indexador *loc*.

In [ ]:
# accede al primer elemento de la week1
print(df_two_weeks.loc[('week1', 0)])

In [ ]:
# accede al primer elemento de la week1, luego Customer ID
print(df_two_weeks.loc[('week1', 0), 'Customer ID'])

Si queremos volver a la vista de un DataFrame con un solo indice y tener las semanas almacenadas podemos usar el metodo *reset_index*.

In [ ]:
# resetamos el indice de nivel 1 numerico
df_two_weeks = df_two_weeks.reset_index(level=1, drop=1)
# pasamos week a columna
df_two_weeks.reset_index(inplace=True)
# renombramos columna de semana de index a week
df_two_weeks.rename(columns={'index':'week'}, inplace=True)
df_two_weeks.head()

## El método `append`

El metodo *append* es como *concat* solo que se llama desde un DataFrame.

In [ ]:
df_two_weeks = df_week1.append(df_week2, ignore_index=True)
df_two_weeks.head()

Cual se debe usar?. Pues todo depende de tu implementacion. __Un detalle a apuntar es que el metodo *merge* no contempla la posibilidad de pasar un parametro *keys* que nos permita saber en este caso de que semana vienen los datos.__

## Realizando merging


## `Inner Joins`

Las operaciones de *merging* (normalmente llamadas *join*) de varias fuentes de datos nos van a permitir extraer informacion de nuestra base de datos para su posterior analisis. Los joins nos van a permitir juntar datasets en base a una *key* que tengan en comun. Por ejemplo, en nuestro caso los ficheros de ventas con el de clientes y comida a traves de sus IDs. [Pulsa aqui para obtener informacion de los distintos joins existentes.](https://stackoverflow.com/questions/38549/what-is-the-difference-between-inner-join-and-outer-join)

Cuando realizamos un *inner join* lo que Pandas hace es buscar claves que sean iguales en dos DataFrames y los junta (**puedes verlo como la interseccion de los datos en base a una columna**) . Por ejemplo imaginemos que queremos saber que clientes comieron una semana en nuestro restaurante. En nuestro fichero clientes estan todos los clientes que han pasado por el restaurante y no todos ellos comieron la semana pasada en el.

In [ ]:
df_week1.head()

In [ ]:
df_customers.head()

In [ ]:
df_week1.merge(df_customers, how='inner', left_on='Customer ID', right_on='ID').head()

Como puedes observar se ha realizado el *merging* de los dos DataFrames. Algunas cosas a comentar:
- how --> indica que tipo de operacion join se va a realizar
- left_on --> indica la key utilizada para unir en el DataFrame de la izquierda
- right_on --> indica la key utilizada para unir en el DataFrame de la derecha

Basicamente en la orden anterior le estamos diciendo a Pandas que nos haga un *join* de los dos Dataframes (week1 y customers) en base a las columnas _Customer ID_ (en week) y ID (en customers). El resultado es al intersercion de ambos DataFrames y por tanto, la informacion de los clientes que vinieron a comer al restaurante la semana 1.

Si queremos hacer lo mismo con la comida?.

In [ ]:
df_week1.merge(df_foods).head()

**NOTA: puedes ver que no utilizamos los parametros *left_on* y *right_on*. Esto es porque los dos DataFrames tienen las columnas que utilizamos para unir con el mismo nombre. Pandas por defecto busca columnas con el mismo nombre y realiza la union a traves de ellas**

## `Outer Joins`

La operacion de outer join la podemos ver como si fuera **la union de los datos en base a una o varias columnas**. Imaginemos que queremos saber cuantas veces cada uno de nuestros clientes vinieron la semana pasada a nuestro restaurante. Si realizamos un *inner join* perderemos a los clientes que la semana pasada no comieron en el restaurante. Si you quiero que estos aparezcan para poder contabilizar que no vinieron podemos hacerlo como un outer join.

In [ ]:
df_customers.merge(df_week1, how='outer', right_on='Customer ID', left_on='ID').head()

Que ha pasado??. Al realizar la union hay clientes que no estan en las ventas de la semana porque no comieron en el restaurante. Estos aparecen como *NaN* en las columnas del DataFrame de ventas (week1) porque no existen registros para ellos. Si pasamos el parametro _indicator_ nos permite averiguar que informacion comparten y existe en cada uno de los DataFrames de forma exclusiva.

In [91]:
# indicator crea una columna llamada _merge que nos permite saber que IDs comparter y cuales
# no los dos DataFrames para los que hacemos el outer join
merged = df_customers.merge(df_week1, how='outer', 
                            right_on='Customer ID', 
                            left_on='ID', 
                            indicator = True)

In [ ]:
# podemos calcular de forma sencilla que IDs hay en:
# df_customers solo : left_only
# en ambos DataFrames : both
# en df_week1 solo : right_only
merged["_merge"].value_counts()

In [ ]:
# IDs que solo estan (NO comparten) en el DataFrame de clientes o de ventas semanal
# basicamente son aquellos clientes que no vinieron a comer esa semana
mask = merged["_merge"].isin(["left_only", "right_only"])
merged[mask].head()

In [ ]:
# IDs que estan (comparten) en el DataFrame de clientes o de ventas semanal
# clientes que vinieron a comer esa semana
mask = merged["_merge"].isin(["both"])
merged[mask].head()

## `Left Joins`

Cuando realizamos un _left join_ Pandas guardara toda la informacion del DataFrame de la izquierda (inclusive la que no pudo enlazar a traves de la clave) al realizar la union. En este caso, la informacion del DataFrame de la derecha que no se enlace es descartada. Podemos utilizarlo por ejemplo para obtener informacion de la comida que se vendio.

In [ ]:
df_week1.head(3)

In [ ]:
df_foods.head(3)

In [ ]:
# Realizamos un left join, y a la vez a traves del parametro sort ordenamos el DataFrame
# en base al ID utilizado para la union (Food ID)
# si existen Food IDs que no estan registrados en nuestro dataset de comida (df_foods)
# apareceran despues del left join con las columnas Food Item y Price como Nans
df_sales_week1_food = df_week1.merge(foods, how = "left", on = "Food ID", sort = True)

Vamos a obtener los nombres de los alimentos y su precio consumidos en la semana 1 de forma ordenada

In [ ]:
df_sales_week1_food.head()

## Merging en base a Indexes con los parámetros `left_index` y `right_index`

Se pueden realizar las operaciones basadas en *index* en vez de columnas. Esto se realiza a traves de los parametros *right_index* y *left_index*. Pandas realizara el *join* utilizando los indices en vez de las columnas. Veamos un ejemplo.

In [ ]:
# cargamos customers y le ponemos customer ID como index
df_customers = pd.read_csv("pandas/Restaurant - Customers.csv", index_col="ID")
# cargamos foods y le ponemos food ID como index
df_foods = pd.read_csv("pandas/Restaurant - Foods.csv", index_col="Food ID")

In [ ]:
df_customers.head(3)

In [ ]:
df_foods.head(3)

In [ ]:
df_week1.head(3)

In [ ]:
# usamos right_index y Pandas une contra el indice de df_customers
sales = df_week1.merge(df_customers, how = "left", left_on = "Customer ID", right_index = True)

In [113]:
# usamos right_index y Pandas une contra el indice de df_foods
sales = sales.merge(df_foods, how = "left", left_on = "Food ID", right_index = True)

In [ ]:
# hemos obtenido informacion de los clientes y las comidas que consumienron
sales.head()

Para obtener mas informacion de los metodos y su funcionamiento puedes acudir a la [documentacion de los mismos](https://pandas.pydata.org/pandas-docs/stable/merging.html).